In [ ]:
# CELL 1: Install + Interactive Login (NO TOKEN IN CODE)
!pip install -q transformers torch accelerate bitsandbytes huggingface_hub

from huggingface_hub import login
import torch

# Interactive login — PASTE TOKEN WHEN PROMPTED
print("PASTE YOUR HUGGING FACE TOKEN BELOW (WILL NOT BE SAVED):")
login()

print("Logged in to Hugging Face!")
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 41.4 MB/s eta 0:00:00
PASTE YOUR HUGGING FACE TOKEN BELOW (WILL NOT BE SAVED):


Logged in to Hugging Face!
GPU: NVIDIA A100-SXM4-80GB


In [ ]:
# CELL 2: Load Llama-3-8B (AFTER LICENSE ACCEPTED)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

print("Loading Llama-3-8B-Instruct... (1-2 minutes)")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    torch_dtype=torch.float16
)

print("Llama-3-8B loaded successfully!")

Loading Llama-3-8B-Instruct... (1-2 minutes)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Llama-3-8B loaded successfully!


In [ ]:
# CELL 3: Fin-R1 CoT Reasoning Test
def fin_r1_query(query):
    prompt = f"""<think>
You are Fin-R1, a financial reasoning LLM. Use Chain-of-Thought.
Reason step by step using financial principles.
</think>
<answer>
Query: {query}
Answer: """

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.7,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract <answer> part
    if "<answer>" in full_response:
        answer = full_response.split("<answer>")[1].strip()
        if "</answer>" in answer:
            answer = answer.split("</answer>")[0].strip()
        return answer
    return full_response.split("Answer:")[-1].strip()

# TEST 1: NPV (Financial Math)
print("TEST 1: NPV Calculation")
print(fin_r1_query("What is the NPV of $10,000 invested at 8% annual rate for 5 years?"))
print("\n" + "-"*50 + "\n")

# TEST 2: Stock Logic
print("TEST 2: Stock Reasoning")
print(fin_r1_query("Should I buy AAPL if P/E is 35 and VIX is 22?"))

TEST 1: NPV Calculation
Query: What is the NPV of $10,000 invested at 8% annual rate for 5 years?
Answer: 1. I understand the problem. We are asked to calculate the present value of $10,000 invested at 8% annual rate for 5 years.

2. I will start by calculating the future value of the investment using the formula: FV = PV * (1 + r)^n, where FV is the future value, PV is the present value ($10,000), r is the annual rate (8%), and n is the number of years (5).

FV = $10,000 * (1 + 0.08)^5 = $13,382.22

3. To calculate the NPV, I will discount the future value back to the present using the formula: NPV = FV / (1 + r)^n. This is because NPV is the present value of the future cash flows.

NPV = $13,382.22 / (1 + 0.08)^5 = $10,000

4. Therefore

--------------------------------------------------

TEST 2: Stock Reasoning
Query: Should I buy AAPL if P/E is 35 and VIX is 22?
Answer:  (After reasoning)


In [ ]:
# CELL 4: Save Locally + Download (NO DRIVE NEEDED)
save_path = "/content/fin-r1-llama3-day1"

# Save model
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model saved locally: {save_path}")

# Zip and download
!zip -r fin-r1-llama3-day1.zip {save_path}

print("Zipped! Click below to download:")
from google.colab import files
files.download('fin-r1-llama3-day1.zip')

Model saved locally: /content/fin-r1-llama3-day1
  adding: content/fin-r1-llama3-day1/ (stored 0%)
  adding: content/fin-r1-llama3-day1/model-00002-of-00002.safetensors (deflated 23%)
  adding: content/fin-r1-llama3-day1/special_tokens_map.json (deflated 61%)
  adding: content/fin-r1-llama3-day1/tokenizer_config.json (deflated 96%)
  adding: content/fin-r1-llama3-day1/chat_template.jinja (deflated 52%)
  adding: content/fin-r1-llama3-day1/tokenizer.json (deflated 85%)
  adding: content/fin-r1-llama3-day1/config.json (deflated 55%)
  adding: content/fin-r1-llama3-day1/model.safetensors.index.json (deflated 96%)
  adding: content/fin-r1-llama3-day1/generation_config.json (deflated 31%)
  adding: content/fin-r1-llama3-day1/model-00001-of-00002.safetensors (deflated 7%)
Zipped! Click below to download:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# CELL 5: CORRECT INSTALL ORDER + RESTART FIX
!pip install -q --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth"
!pip install -q --no-deps trl peft accelerate bitsandbytes

# Import unsloth FIRST
from unsloth import FastLanguageModel
import torch

print("Unsloth loaded successfully! (No xformers, no datasets yet)")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth loaded successfully! (No xformers, no datasets yet)


In [ ]:
# ALTERNATIVE CELL 6: Load Llama-3-8B-Instruct directly from HF (NO ZIP!)
from unsloth import FastLanguageModel
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

print("Loading Llama-3-8B-Instruct from Hugging Face... (5–10 minutes)")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    dtype=None,
    load_in_4bit=True,
    device_map="auto"
)

# Enable faster inference
FastLanguageModel.for_inference(model)

print("Llama-3-8B loaded successfully! Ready for SFT.")

Loading Llama-3-8B-Instruct from Hugging Face... (5–10 minutes)
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

Llama-3-8B loaded successfully! Ready for SFT.


In [ ]:
 # CELL: Install Datasets (if not done)
!pip install -q datasets

print("Datasets library ready!")

Datasets library ready!


In [ ]:
# CELL 7: Upload ANY ZIP files (no name restriction)
from google.colab import files

print("UPLOAD YOUR FINQA ZIP FILE (any name)...")
uploaded = files.upload()  # Upload first ZIP

# Unzip (works with any name)
for filename in uploaded.keys():
    !unzip -o "{filename}"
    print(f"{filename} extracted!")

print("\nUPLOAD YOUR CONVFINKA ZIP FILE (any name)...")
uploaded = files.upload()  # Upload second ZIP

for filename in uploaded.keys():
    !unzip -o "{filename}"
    print(f"{filename} extracted!")

print("Both datasets ready!")

UPLOAD YOUR FINQA ZIP FILE (any name)...


Saving finqa_data.zip to finqa_data.zip
Archive:  finqa_data.zip
  inflating: dev.json                
  inflating: test.json               
  inflating: train.json              
finqa_data.zip extracted!

UPLOAD YOUR CONVFINKA ZIP FILE (any name)...


Saving conv_train.zip.zip to conv_train.zip.zip
Archive:  conv_train.zip.zip
cf3eed2d5984960bf06bb8145bcea5e80b0222a6
   creating: ConvFinQA-main/
  inflating: ConvFinQA-main/LICENSE  
  inflating: ConvFinQA-main/README.md  
   creating: ConvFinQA-main/code/
   creating: ConvFinQA-main/code/finqanet_generator/
  inflating: ConvFinQA-main/code/finqanet_generator/Convert.py  
  inflating: ConvFinQA-main/code/finqanet_generator/Main.py  
  inflating: ConvFinQA-main/code/finqanet_generator/Model_new.py  
  inflating: ConvFinQA-main/code/finqanet_generator/Test.py  
   creating: ConvFinQA-main/code/finqanet_generator/__pycache__/
  inflating: ConvFinQA-main/code/finqanet_generator/__pycache__/DataLoader.cpython-37.pyc  
  inflating: ConvFinQA-main/code/finqanet_generator/__pycache__/Model.cpython-37.pyc  
  inflating: ConvFinQA-main/code/finqanet_generator/__pycache__/Model_new.cpython-37.pyc  
  inflating: ConvFinQA-main/code/finqanet_generator/__pycache__/config.cpython-37.pyc  
  inflati

In [ ]:
# CELL 7.5: Unzip ConvFinQA's data.zip
!unzip -o ConvFinQA-main/data.zip -d ConvFinQA-main/

print("ConvFinQA data extracted!")
!ls ConvFinQA-main/

Archive:  ConvFinQA-main/data.zip
   creating: ConvFinQA-main/data/
  inflating: ConvFinQA-main/data/test_private.json  
  inflating: ConvFinQA-main/data/train.json  
  inflating: ConvFinQA-main/data/dev_turn.json  
  inflating: ConvFinQA-main/data/test_turn_private.json  
  inflating: ConvFinQA-main/data/train_turn.json  
  inflating: ConvFinQA-main/data/dev.json  
ConvFinQA data extracted!
code  data  data.zip  LICENSE  README.md


In [ ]:
# CELL 8 – FINAL WORKING: Load FinQA + ConvFinQA → CoT Dataset (NO ERRORS)

import json
from datasets import Dataset

# ------------------------------------------------------------------
# 1. Load raw files
# ------------------------------------------------------------------
print("Loading FinQA train.json...")
with open("train.json", "r") as f:
    finqa_raw = json.load(f)
print(f"FinQA examples loaded: {len(finqa_raw)}")

print("Loading ConvFinQA train_turn.json...")
with open("ConvFinQA-main/data/train_turn.json", "r") as f:
    conv_raw = json.load(f)

# ------------------------------------------------------------------
# 2. Normalize ConvFinQA (list or dict)
# ------------------------------------------------------------------
if isinstance(conv_raw, list):
    convfinqa = conv_raw
    print("ConvFinQA: loaded as list")
elif isinstance(conv_raw, dict):
    keys = [k for k, v in conv_raw.items() if isinstance(v, list)]
    if not keys:
        raise ValueError("ConvFinQA dict has no list field")
    convfinqa = conv_raw[keys[0]]
    print(f"ConvFinQA: extracted from key '{keys[0]}'")
else:
    raise TypeError("ConvFinQA must be list or dict")

print(f"ConvFinQA examples: {len(convfinqa)}")

# ------------------------------------------------------------------
# 3. Helper: safe join
# ------------------------------------------------------------------
def safe_join(items, sep=" "):
    return sep.join(str(x) for x in items if x is not None)

# ------------------------------------------------------------------
# 4. FinQA → CoT (answer is in qa.answer)
# ------------------------------------------------------------------
def finqa_to_cot(ex):
    try:
        pre = safe_join(ex.get("pre_text", []))
        post = safe_join(ex.get("post_text", []))
        table = ex.get("table", [])
        table_str = " | ".join([" ".join(map(str, row)) for row in table[:3]]) if table else ""

        # Answer is in qa.answer
        answer = ex["qa"]["answer"]
        reasoning = safe_join(ex["qa"].get("reasoning", []))

        think = f"{pre} {post}"
        if table_str:
            think += f" Table: {table_str}"
        if reasoning:
            think += f" Reasoning: {reasoning}"

        return {"text": f"<think>{think.strip()}</think><answer>{answer}</answer>"}
    except Exception as e:
        print(f"[FinQA SKIP] {e} | Keys: {list(ex.keys())} | qa keys: {list(ex.get('qa', {}).keys())}")
        return None

# ------------------------------------------------------------------
# 5. ConvFinQA → CoT (answer in last turn)
# ------------------------------------------------------------------
def convfinqa_to_cot(ex):
    try:
        turns = ex.get("turns", [])
        if not turns:
            return None
        context = safe_join([t.get("utterance", "") for t in turns])
        answer = turns[-1].get("answer", "")
        if not answer:
            return None
        return {"text": f"<think>{context.strip()}</think><answer>{answer}</answer>"}
    except Exception as e:
        print(f"[ConvFinQA SKIP] {e}")
        return None

# ------------------------------------------------------------------
# 6. Build dataset
# ------------------------------------------------------------------
print("Processing FinQA...")
finqa_cot = [x for x in map(finqa_to_cot, finqa_raw) if x is not None]

print("Processing ConvFinQA...")
conv_cot = [x for x in map(convfinqa_to_cot, convfinqa) if x is not None]

all_cot = finqa_cot + conv_cot
print(f"\nTOTAL VALID CoT EXAMPLES: {len(all_cot)}")

# ------------------------------------------------------------------
# 7. Create Hugging Face Dataset
# ------------------------------------------------------------------
if len(all_cot) == 0:
    raise ValueError("No valid CoT examples found. Check your JSON structure.")

cot_dataset = Dataset.from_list(all_cot)

# ------------------------------------------------------------------
# 8. SAFE PREVIEW (no IndexError)
# ------------------------------------------------------------------
print("\nDataset created successfully!")
print("First example preview:")
print(cot_dataset[0]["text"][:700] + "..." if len(cot_dataset[0]["text"]) > 700 else cot_dataset[0]["text"])

# ------------------------------------------------------------------
# 9. Sanity check
# ------------------------------------------------------------------
sample = cot_dataset[0]["text"]
assert "<think>" in sample and "<answer>" in sample, "Missing tags!"
print("\nTags <think> and <answer> confirmed.")

Loading FinQA train.json...
FinQA examples loaded: 6251
Loading ConvFinQA train_turn.json...
ConvFinQA: loaded as list
ConvFinQA examples: 11104
Processing FinQA...
Processing ConvFinQA...

TOTAL VALID CoT EXAMPLES: 6251

Dataset created successfully!
First example preview:
<think>interest rate to a variable interest rate based on the three-month libor plus 2.05% ( 2.05 % ) ( 2.34% ( 2.34 % ) as of october 31 , 2009 ) . if libor changes by 100 basis points , our annual interest expense would change by $ 3.8 million . foreign currency exposure as more fully described in note 2i . in the notes to consolidated financial statements contained in item 8 of this annual report on form 10-k , we regularly hedge our non-u.s . dollar-based exposures by entering into forward foreign currency exchange contracts . the terms of these contracts are for periods matching the duration of the underlying exposure and generally range from one month to twelve months . currently , our...

Tags <think> and <an